패키지 삽입

In [1]:
import sys, os
import numpy as np
import mujoco
from mujoco import viewer
import mediapy as media
import matplotlib.pyplot as plt

Gait 객체 정의

In [2]:
class snake_gait:
    def __init__(self, gait_type: int = 1, start_k: int = 0):
        self.k = start_k

        if gait_type == 0: #Vertical
            self.M = np.array([[1,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0],
                                [0,1,0,0,0,0,0],
                                [0,0,0,0,0,0,0],
                                [0,0,1,0,0,0,0],
                                [0,0,0,0,0,0,0],
                                [0,0,0,1,0,0,0],
                                [0,0,0,0,0,0,0],
                                [0,0,0,0,1,0,0],
                                [0,0,0,0,0,0,0],
                                [0,0,0,0,0,1,0],
                                [0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,1],
                                [0,0,0,0,0,0,0]],dtype='int')

        elif gait_type == 1: #Serpentine
            self.M = np.eye(14)

        elif gait_type == 2: #Sidewinding
            self.M = np.array([[0,1,0,0,0,0,0,0,0,0,0,0,0,0],
                                [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
                                [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
                                [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                                [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
                                [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
                                [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
                                [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
                                [0,0,0,0,0,0,0,0,0,0,0,0,1,0]],dtype='int')

        else:
            self.M = np.ones((14,1))

        self.M_cols = self.M.shape[1]

        self.k = self.k % self.M_cols

    def get_selected_joints(self) -> np.ndarray:
        return self.M[:,self.k]

    def gait_step(self):
        self.k = (self.k + 1) % self.M_cols

시뮬레이션 정의 코드 작성

In [3]:

filename = ".\\description\\snake_circle.xml"
# dirpath = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(__file__))) + "\\description\\"

path = filename

model = mujoco.MjModel.from_xml_path(path)
renderer = mujoco.Renderer(model, 480, 480)
data = mujoco.MjData(model)

mujoco.mj_forward(model, data)
renderer.update_scene(data, "fixed")

gait = snake_gait()
frames = []

for t in range(1000):
    u = gait.get_selected_joints() * (np.random.random((1,1)) - 0.5) * 6
    u = u.squeeze()
    idx = int(np.nonzero(u)[0])
    gait.gait_step()

    data.ctrl[idx] = u[idx]

    gait.gait_step()
    mujoco.mj_step(model,data)

    # print(data.body("head"))
    pixels = renderer.render().copy()
    frames.append(pixels)

media.show_video(frames, fps=100)

RuntimeError: Program 'ffmpeg' is not found; perhaps install ffmpeg using 'apt-get install ffmpeg'.

시뮬레이션 시작